# Import software libraries and load the dataset #

In [ ]:
import sys                                             # Read system parameters.
import os                                              # Interact with the operating system.
import numpy as np                                     # Work with multi-dimensional arrays and matrices.
import pandas as pd                                    # Manipulate and analyze data.
import matplotlib as mpl                               # Create 2D charts.
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sb                                   # Perform data visualization.
import sklearn                                         # Perform data mining and analysis.
from time import time                                  # Calculate training time.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- NumPy {}'.format(np.__version__))
print('- pandas {}'.format(pd.__version__))
print('- Matplotlib {}'.format(mpl.__version__))
print('- scikit-learn {}\n'.format(sklearn.__version__))

# Load the dataset.
PROJECT_ROOT_DIR = '.'
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, 'power_plant_data')
print('Data files in this project:', os.listdir(DATA_PATH))
data_raw_file = os.path.join(DATA_PATH, 'cc_power_plant_data.csv')
data_raw = pd.read_csv(data_raw_file)
print('Loaded {} records from {}.'.format(len(data_raw), data_raw_file))

# Get acquainted with the dataset

In [ ]:
print(data_raw.info())     # View features and data types.
data_raw.head(10)          # View first 10 records.

# Examine the distribution of the features

In [ ]:
# Use Matplotlib to plot figures.
%matplotlib inline
mpl.rc('axes', labelsize = 14)
mpl.rc('xtick', labelsize = 12)
mpl.rc('ytick', labelsize = 12)

data_raw.hist(figsize = (8, 8));
plt.figure();

# Examine a general summary of statistics

In [ ]:
with pd.option_context('float_format', '{:.2f}'.format): 
    print(data_raw.describe())

# Look for columns that correlate with `EnergyOutput`#

In [ ]:
# Correlations between numeric features and 'EnergyOutput'.
print('Correlations with EnergyOutput')
print(data_raw.corr()['EnergyOutput'].sort_values(ascending=False))

# Split the datasets

In [ ]:
from sklearn.model_selection import train_test_split

# 'EnergyOutput' is the dependent variable (value to be predicted), so it will be
# removed from the training data and put into a separate DataFrame for labels.
label_columns = ['EnergyOutput']

training_columns = ['Temperature', 'ExhaustVacuum', 'AmbientPressure', 'RelativeHumidity']

# Split the training and test datasets and their labels.
X_train, X_test, y_train, y_test = train_test_split(data_raw[training_columns],
                                                                            data_raw[label_columns],
                                                                            random_state = 543)

# Compare the number of rows and columns in the original data to the training and test sets.
print(f'Original set:        {data_raw.shape}')
print('------------------------------')
print(f'Training features:   {X_train.shape}')
print(f'Test features:       {X_test.shape}')
print(f'Training labels:     {y_train.shape}')
print(f'Test labels:         {y_test.shape}')

# Create a linear regression model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression(fit_intercept = False)
start = time()
lin_reg.fit(X_train, np.ravel(y_train))
end = time()
train_time = (end - start) * 1000

# Score using the test data.
score = lin_reg.score(X_test, y_test)

print('Linear regression model took {:.2f} milliseconds to fit.'.format(train_time))
print('Variance score on test set: {:.0f}%'.format(score * 100))

# Compare the first ten predictions to actual values

In [ ]:
predict = lin_reg.predict(X_test)

results_comparison = X_test.copy()
results_comparison['PredictedEnergyOutput'] = np.round(predict, 2)
results_comparison['ActualEnergyOutput'] = y_test.copy()

# View examples of the predictions compared to actual energy output.
results_comparison.head(10)

# Calculate the error between predicted and actual values

In [ ]:
from sklearn.metrics import mean_squared_error as mse

cost = mse(y_test, predict)
print('Cost (mean squared error): {:.2f}'.format(cost))

# Plot lines of best fit for all features

In [ ]:
# Prepare four plots showing lines of best fit for energy output and other features.
line_color = {'color': 'red'}
fig, ax = plt.subplots(2, 2, figsize = (18, 10))

# Temperature
ax1 = sb.regplot(X_test['Temperature'], 
                np.ravel(predict), 
                line_kws = line_color, 
                ax = ax[0,0])
ax1.set_ylabel('Energy Output')

# Exhaust vacuum
ax2 = sb.regplot(X_test['ExhaustVacuum'],
                 np.ravel(predict), 
                 line_kws = line_color, 
                 ax = ax[0,1])
ax2.set_ylabel('Energy Output')

# Ambient pressure
ax3 = sb.regplot(X_test['AmbientPressure'], 
                 np.ravel(predict), 
                 line_kws = line_color,
                 ax = ax[1,0])
ax3.set_ylabel('Energy Output')

# Relative humidity
ax4 = sb.regplot(X_test['RelativeHumidity'],
                 np.ravel(predict), 
                 line_kws = line_color, 
                 ax = ax[1,1])
ax4.set_ylabel('Energy Output')

ax4.set_ylim(420,498)
ax4.set_xlim(0,110);

# Compare predicted values to actual values

In [ ]:
N = 10  # Plot every Nth value to save time and space
predict_df = results_comparison.sort_values('ActualEnergyOutput')[::N]

predict_df['diff'] = predict_df['ActualEnergyOutput'] - predict_df['PredictedEnergyOutput']
predict_df['recnum'] = np.arange(len(predict_df))
predict_df['error_pct'] = abs(predict_df['diff'] / predict_df['ActualEnergyOutput']) * 1500

ax = plt.figure(figsize = [18, 10])
plt.ylabel('Energy Output')
plt.xlabel('Measurement')
plt.plot(predict_df['recnum'], predict_df['ActualEnergyOutput'], color = 'blue');
plt.scatter(predict_df['recnum'],
            predict_df['PredictedEnergyOutput'], 
            predict_df['error_pct'], 
            color = 'red');

ax.legend(['Actual', 'Predicted'], 
           loc = 'lower center',
           ncol = 2, 
           title = 'Energy output predicted using linear regression')

plt.show()

# Retrieve the model parameters

In [ ]:
coefs = lin_reg.coef_

print('Temperature coefficient: {}'.format(coefs[0]))
print('ExhaustVacuum coefficient: {}'.format(coefs[1]))
print('AmbientPressure coefficient: {}'.format(coefs[2]))
print('RelativeHumidity coefficient: {}'.format(coefs[3]))

# Manually calculate the model parameters using the normal equation

In [ ]:
# Multiply the transpose of X by X, then take the inverse.
eq_part_1 = np.linalg.inv(X_train.transpose().dot(X_train))

# Multiply the transpose of X by y.
eq_part_2 = X_train.transpose().dot(y_train)

# Multiply the previous products.
model_params = eq_part_1.dot(eq_part_2)

print(model_params)